
# Objectives
- Interacting atomically with two Dapps
- Interacting with various types of on-chain exchanges as a trader
- Hands-on experience using Uniswap V3 as an LP

# Prerequisites 
- Metamask extension should be installed on the browser, if not, install Metamask from [here](https://metamask.io/).
- Sign up for an HTTP web3 provider service such as Infura. The function of a provider is to answer queries about the state of the contract that can be later used to make a trade and post transactions containing the swaps onto the testnet.

# Interacting atomically with two Dapps


## Hotel and Flight contracts

Assume you are going on a vacation and need to book travel and accommodation. We have created two contracts mimicking a decentralized application for booking accommodation and travel. The hotel reservation contract can be found [here](https://sepolia.etherscan.io/address/0x93189d3749f61bbd662ac0bfe3ff7f37a48110aa). Let us go through its main ``bookRoom()`` function

- Initial few checks ensure the payment is sufficient and rooms are available at the hotel
- The state of the contract is updated to make sure the room is booked in the name of the wallet initiating the transaction calling the function
- The last part calculates and disburses a refund if the transaction caller sent more money than the ticket price of `10**10 wei`.
- Note that no part of the contract allows for cancellation of the reservation; this Dapp corresponds to a Non-refundable reservation

You will now book accommodation for the trip. 
- Go to the contract on Etherscan, click the write contract tab, and connect your wallet. - - Click the [bookRoom()](https://sepolia.etherscan.io/address/0x66e6ccf3f575cdb27ce574fd49d74c8b274cd21d#writeContract#F2) tab. 
- Set the amount of Wei to send as a payment. The required payment is `10000000000 wei` or `0.00000001 ETH` (recommend); you can send more if you want, and the contract will refund it within the same transaction.
- Check the status of your reservation by querying `CheckReservations` with your public key

Now that we have booked the accommodation, let us book a flight. The flight reservation contract looks similar to the hotel reservation contract with ``bookFlight()`` replacing ``bookRoom()``. You can access the flight reservation contract [here](https://sepolia.etherscan.io/address/0x27dD8de8c733c546FeEa68bE6CeFBFFb7Bb8A219). Perform the same steps as the hotel reservation contract to book a flight ticket. Check the status of the transaction on Etherscan.

Notice that the transaciton will fail for some of you since there aren't enough flight tickets available for all class members. This leads to a situation where your hotel reservation is practically useless since there is no way to travel. Is it possible to avoid such a situation: *Book hotel only if the flight is available* and *Book flight only if the hotel is available*? In other words, we want the trip reservation to be atomic - Either book both travel and accommodation or book none. Notice that the Dapps are independent of one another - think of them as two separate travel websites. 

## Atomic transaction

We can perform such transactions easily on blockchains. Note that an ethereum transaction either succeeds or fails and reverts back. Thus, we need to combine ``bookRoom()`` and ``bookFlight()`` into a single transaction. 

We can do this by launching a smart contract whose one function call will call two of the above hotel and flight contracts. The atomic transaction contract (aka trip reservation contract) can be found [here](https://sepolia.etherscan.io/address/0xa67990332E026b6c4AC71937BbE44207d97c113a). Let us quickly go through the ``bookTrip`` function:
- The function takes in 4 arguments - the addresses of Hotel and Flight reservation contracts and their prices
- It then calls both the hotel and flight contract 
- The calls are sequential, however, due to the atomic nature of an Ethereum transaction, the whole ``bookTrip`` transaction will fail if either of the contract calls fails.
- The last part disburses any remaining refund.

I have refreshed both the hotel and flight contracts, and you will now use the atomic transaction contract to book both atomically:
- Go to the contract on Etherscan, click the write contract tab, and connect your wallet. - - Click the [bookTrip](https://sepolia.etherscan.io/address/0xa67990332E026b6c4AC71937BbE44207d97c113a#writeContract#F1) tab. 
- Set the amount of ETH to send as a payment. The required total payment is `20000000000 wei` or `0.00000002 ETH`; anything extra will be refunded. 
- Give the contract addresses of the hotel and flight reservation contract; set the price of both to ``10000000000`` wei.
- Check the status of the transaction on Etherscan

The transaction will succeed for some of you and fail for the rest. Check the status of your booking on the respective contracts. If the atomic transaction failed, you should not have confirmed reservations on both the hotel and flight contracts. 


# Trading on decentralized exchange applications

Decentralized exchanges can be categorized into two broad categories:
- Dapps mimicking traditional centralized exchanges and implementing an order book
- Dapps running DeFi native exchange formats such as AMMs
In this lab, we will interact with one exchange of each of the above types through their web app.

## Trade on an order book exchange


Order book exchange enables trading by running sequential trades on an order book. The order book contains *bids* from agents who want to buy an asset, and *asks* from agents who want to sell the asset. A trade occurs when a bid matches an ask. One can perform two primary types of orders on such an exchange. 
- A *market order* is an order that executes the trade at the best available bid or asks.
- A *limit order* that sets a bid or ask on an order book that is executed at a price within limits set by the order.

In this lab, we will interact with dYdX V4, an order book DEX implemented on the dYdX Chain testnet. 

- Open [dYdX V4 testnet app](https://v4.testnet.dydx.exchange/#/trade/ETH-USD)
- Click on the top-right corner to connect your metamask wallet 
- After the connection, click *Deposit funds* (or *deposit*)

We will now be performing a market order. Please follow the steps below: 
- Select market order on the right
- Trade 10 USD for ETH - Click on sell and set 10 USD
- Notice that you can choose the Leverage as well (here, we use the default)
- Click *Place Market Order* 
- Observe the change of Equity and Buying Power on the top-right
- Turn to Fills, check the information such as type, side, amount, price

We will now perform a limit order. Please follow the steps below:
- Select limit order on the right
- Observe the Price of USD for both *bids* and *asks*
- Set a price between the lowest of asks and the highest of bids as the Limit Price (fill into the Limit Price)
- Trade 10 USD for ETH - Click on sell and set 10 USD
- Set other parameters as you want (here, we use the default)
- Click *Place Limit Order* 
- Turn to Orders to see the order in process, observe the change of *asks*
- After finishing, turn to Fills to check the information

## Trade on an AMM

We will be contrasting the above trading experience with Uniswap. Uniswap is an AMM; its implementation is lightweight by design; hence, it can be implemented entirely on an L1 blockchain such as Ethereum. As a contract with dYdX V4, trading on Uniswap does not require an account since it interacts directly with the wallet. We have created a trading pair to exchange HUSD-PEARL. Let us perform a simple HUSD-PEARL trade on Uniswap.

- Open the Uniswap app and select swap at [https://app.uniswap.org/#/swap](https://app.uniswap.org/#/swap)
- Connect your metamask wallet on the top right
- Check HUSD and PEARL funds on metamask
- If metamask doesn't show the HUSD/PEARL token; we need to import it (HUSD: 0xe41Fa6BF04aAF0dD6E44d62b1A1Bd8209dc06f69, PEARL: 0x3d1544B6FecE99C62a0f14D32759eb2cCE183670)
- Paste the HUSD contract address on the from tab (0xe41Fa6BF04aAF0dD6E44d62b1A1Bd8209dc06f69)
- Paste the PREAL contract address on the to tab (0x3d1544B6FecE99C62a0f14D32759eb2cCE183670)
- You would need to approve HUSD tokens to be used by the Uniswap contract; click on approve and confirm on Metamask.
- Once approval is complete, swap 10 HUSD by clicking on the swap tab and confirming the transaction on Metamask
- Click on swap and confirm the transaction on metamask
- Select the txid link (on the top-right Pending tab) on Uniswap and observe the trade transaction on Etherscan 
- The trade should show pending and should be successful in less than 30 sec
- Check HUSD and PEARL funds on metamask again



Note that all interactions are visible through your metamask wallet since they occur on the Sepolia blockchain. AMMs like Uniswap also allow anyone to create their trading pair, enabling exchanges for long-tail assets such as HUSD and PEARL.


# Interacting with Uniswap as a Liquidity provider

Uniswap allows anyone to provide liquidity to any token pair. Depositing tokens in a token-pair's liquidity pool enables that entity to own a pool and get the exchange fee rewards. In this lab, we will provide liquidity to the HUSD-PEARL liquidity pool on Uniswap. 

- Open Uniswap app for adding liquidity at [liquidity](https://app.uniswap.org/#/add)
- Click on add liquidity
- Select the first token in the pair as HUSD (set address as: 0xe41Fa6BF04aAF0dD6E44d62b1A1Bd8209dc06f69)
- Select the second token in the pair as PEARL (set address as: 0x3d1544B6FecE99C62a0f14D32759eb2cCE183670)
- Set 10 HUSD, which should automatically populate PEARL based on the current state of the bonding curve
- Approve spending HUSD and PEARL
- Wait for the approval transaction to be included in the chain
- Click on supply to add liquidity to the pool
- Observe the transaction on Etherscan - it should state that an ERC-721 token was transferred to your account (labeled Uniswap V3)
- This ERC721 token is an LP token for the HUSD-PEARL liquidity pool
- Check metamask and import the LP pair under assets named as ``Uniswap V3 Positions NFT-V1``
- The position should appear on [https://app.uniswap.org/#/pool](https://app.uniswap.org/#/pool)

The LP token is like any other token; you can transfer it between accounts and have that account claim ownership of the liquidity pool. To check this, create a new wallet account as follows:
- On metamask, click on the 'my accounts' icon on the top right and select ``Create account``
- Give it an appropriate name and add funds to it by going to the [Sepolia faucet](https://sepoliafaucet.com/)

Now we will transfer ownership of the liquidity pool by sending LP tokens to your second account
- Change your account in your metamask to your original account
- Set asset to ``Uniswap V3 Positions NFT-V1``
- Select send, followed by transfer between my accounts, followed by your second account
- Confirm the transaction
- Import ``Uniswap V3 Positions NFT-V1`` token on your second account
- Go to Uniswap and check the status of your Liquidity positions on [https://app.uniswap.org/#/pool](https://app.uniswap.org/#/pool)
- Notice that it does not show any liquidity available for the first account
- Go to metamask and switch to the second account (click on connect)
- Observe that all liquidity is shown in the second account
- Liquidate the positions (click on remove liquidity and set max) and observe the tokens transferred to your account (you may have to import UETH and UHKD to your second metamask account)